# Título  Variables con RF
## Autor: Jose Chelquer
## Fecha de última modificación: 21/11/2024
## Descripción:
Agrega features corriendo RF con lgbm.
Agrega variables por cada hoja del lgbm, indicando si la observación está o no ahi.

Para entrenar y predecir, no usa las variables relacionadas con foto_mes


## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [1]:
prueba=False                     # temporario. Volver a false


experimento="_pc_"                        # 

tipo_registros=experimento[0]            # '_'completo s sampleado
con_preprocesamiento = experimento[1]    # p con preprocesamiento  '_' sin preprocesamiento
tipo_featuring=experimento[2]            # _: 155 base  s: simple    c: completo
rf=experimento[3]                        # _: sin RF   r: con RF

dict_tipo_registros={'_': 'completo', 's': 'sampleado'}
dict_con_preprocesamiento={'_': 'sin preprocesamiento', 'p':'con preprocesamiento'}
dict_tipo_featuring={'_': 'base', 's':' simple', 'c': 'completo'}
dict_rf={'_': 'sin RF', 'f':' con RF'}

print (f'Experimento {experimento}')
print (dict_tipo_registros[tipo_registros])
print (dict_con_preprocesamiento[con_preprocesamiento])
print (dict_tipo_featuring[tipo_featuring])
print (dict_rf[rf])


Experimento _pc_
completo
con preprocesamiento
completo
sin RF


In [2]:
# vm o local?
vm=True
if vm:
  usar_gdrive=False
else:
  usar_gdrive=True      #se va a usar google dirve


In [3]:
semillas = [17,19,23,29,31]
ganancia_acierto=273000
costo_estimulo=7000

In [4]:
# datos de entrenamiento
if vm:
  meses_entrenamiento=[202010, 202011, 202012]
  submuestrear=False
else:
  meses_entrenamiento=[202101, 202102, 202103]
  submuestrear=False

grabar_importancias=False          # Se puede pedir que grabe las importancias de variables como resultado secundario
importancias_file='importancias_rf.csv.gz'


## Input

< Archivos de datos (parquet.gz) con sus paths que van a consumirse por el job>

In [5]:
# El script se adapta a archivos .parquet o .parquet.gz
if vm:
  dataset_path = '/home/jose/buckets/b1/datasets/'
  dataset_file=f'k3_{tipo_registros}{con_preprocesamiento}{tipo_featuring}_.parquet.gz'
else:
  dataset_path='/content/drive/MyDrive/Data Science y similares/Maestría Data Mining Exactas/dmeyf/dmeyf2024/datasets/'
  dataset_file='k3_sample_parquet.gz'

print (dataset_path)
print (dataset_file)

/home/jose/buckets/b1/datasets/
k3__pc_.parquet.gz


## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [6]:
# el script se adapta a datasets .parquet o .gz
if vm:
  output_file=f'k3_{tipo_registros}{con_preprocesamiento}{tipo_featuring}f.parquet.gz'
else:
  output_file='k3_sample.conRF.parquet.gz'


## Procesos

## 

### Paquetes necesarios

In [7]:
#%pip install optuna==3.6.1
%pip install optuna==4.0



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

Instalamos, cargamos y seteamos el entorno

In [8]:
#%pip install scikit-learn==1.3.2
#%pip install seaborn==0.13.1
#%pip install numpy==1.26.4
#%pip install matplotlib==3.7.1



## Gdrive?

In [9]:
if usar_gdrive:
  from google.colab import drive
  drive.mount('/content/drive')

## Librerías

In [10]:
!pip install --upgrade pip
!pip install --upgrade lightgbm
!pip install dask[dataframe]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.


In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import shutil

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample

from joblib import Parallel, delayed

import optuna
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time
from sqlalchemy import create_engine

import lightgbm as lgb
from math import exp

Matplotlib is building the font cache; this may take a moment.
/home/jose/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Leer dataset

In [12]:
data_original = pd.read_parquet(os.path.join(dataset_path, dataset_file))

In [13]:
def contabilizar(df, descripción):
  print (f'\nContabilización de la base {descripción}\n')
  print (f'\nShape: {df.shape}\n')
  print (f"\ por mes: \n{df['foto_mes'].value_counts()}\n")
  print (f"\nFilas por mes y clase: \n{pd.crosstab(df['foto_mes'], df['clase_ternaria'])}\n")
contabilizar(data_original, 'Datos Leídos')

<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:4: SyntaxWarning: invalid escape sequence '\ '
/tmp/ipykernel_2158/3529553369.py:4: SyntaxWarning: invalid escape sequence '\ '
  print (f"\ por mes: \n{df['foto_mes'].value_counts()}\n")



Contabilización de la base Datos Leídos


Shape: (4901237, 1302)

\ por mes: 
foto_mes
202109    165644
202108    165442
202107    165152
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
202012    161526
202011    160742
202010    159731
202009    158371
202008    157058
202007    155764
202006    153757
202005    151261
202004    149872
202003    149356
202002    147109
202001    143966
201912    140661
201911    138667
201910    136682
201909    134314
201908    132664
201907    130724
201906    129186
201905    127659
201904    126996
201903    126436
201902    125799
201901    124752
Name: count, dtype: int64


Filas por mes y clase: 
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
201901             688     718    123346
201902             720     688    124391
201903             688     760    124988
201904             759     579    125658
201905             580     660    126419
201906      

## Recodificar a clase binaria

In [14]:
data_original['clase_peso'] = 1.0

data_original.loc[data_original['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data_original.loc[data_original['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [15]:
data_original['clase_binaria1'] = 0
data_original['clase_binaria2'] = 0
data_original['clase_binaria1'] = np.where(data_original['clase_ternaria'] == 'BAJA+2', 1, 0)
data_original['clase_binaria2'] = np.where(data_original['clase_ternaria'] == 'CONTINUA', 0, 1)

In [16]:
# Elige los meses y balancea
if meses_entrenamiento:
  data = data_original[data_original['foto_mes'].isin(meses_entrenamiento)]
else:
  data = data_original.copy()
# Elimina las variables asociadas a foto_mes
cols_fotos=[col for col in data.columns if 'foto_mes_lag' in col]
data=data.drop(columns=cols_fotos)

#Submuestrear para balancear
if submuestrear:
  # Separar las clases
  data_0 = data[data['clase_binaria2'] == 0]
  data_1 = data[data['clase_binaria2'] == 1]
  n_samples=min(len(data_0), len(data_1))

  # Hacer un submuestreo de la clase mayoritaria
  data_0 = resample(data_0,
                    replace=False,  # No reemplace
                    n_samples=n_samples,  # Igualar tamaño de la clase minoritaria
                    random_state=123)  # Para reproducibilidad
  data_1 = resample(data_1,
                    replace=False,  # No reemplace
                    n_samples=n_samples,  # Igualar tamaño de la clase minoritaria
                    random_state=123)  # Para reproducibilidad

  # Combinar clases balanceadas
  data_balanced = pd.concat([data_0, data_1])

  # Mezclar el DataFrame resultante
  data = data_balanced.sample(frac=1, random_state=123).reset_index(drop=True)

contabilizar (data, 'Datos para hacer RF')

y = data['clase_binaria2']
X=data.drop(['clase_ternaria','clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)




Contabilización de la base Datos para hacer RF


Shape: (481999, 1305)

\ por mes: 
foto_mes
202012    161526
202011    160742
202010    159731
Name: count, dtype: int64


Filas por mes y clase: 
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
202010             565     488    158678
202011             490     646    159606
202012             649     634    160243



## Función ganancia

In [17]:
def rf_gan_eval(y_pred, data):
    clase_ternaria = data.get_clase_ternaria()
    # Diferencia si eran BAJA+1 o BAJA+2
    ganancia = np.where(clase_ternaria == 'BAJA+2', ganancia_acierto, 0) - np.where(clase_ternaria !='BAJA+2', costo_estimulo, 0)
    #Ordena ganancia según los índices ordenados de y_pred de mayor a menor
    ganancia = ganancia[np.argsort(y_pred)[::-1]] #: desde todo : hasta todo :-1 step hacia atrás
    # Ganancias acumuladas so far
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

def ganancia_prob(y_hat, y, prop=1, class_index=1, threshold=0.025):
  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == "BAJA+2" else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop



## Imputar NANs

In [18]:
print (f'Cant de Nans:{X.isnull().sum()}')
#imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
#if X.isnull().values.any():
#  X = imp_mean.fit_transform(X)


Cant de Nans:numero_de_cliente                    0
foto_mes                             0
active_quarter                       0
cliente_vip                          0
internet                             0
                                  ... 
Tarjeta_mpagosdolares_n10_lag2    9900
Tarjeta_mconsumototal_n10_lag1    4664
Tarjeta_mconsumototal_n10_lag2    9900
Tarjeta_mpagominimo_n10_lag1      4664
Tarjeta_mpagominimo_n10_lag2      9900
Length: 1301, dtype: int64


## Ajustar

In [19]:
# Cambio
# Fitear el modelo con X e y
model = lgb.LGBMClassifier(
    # parametros que se pueden cambiar
    num_iterations = 20,
    num_leaves  = 16,
    min_data_in_leaf = 1000,
    feature_fraction_bynode  = 0.2,

    # para que LightGBM emule Random Forest
    boosting_type = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1,
    feature_fraction = 1,

    # genericos de LightGBM
    max_bin = 31,
    objective = "binary",
    first_metric_only = True,
    boost_from_average = True,
    feature_pre_filter = False,
    force_row_wise = True,
    verbosity = -100,
    max_depth = -1,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = True,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = False
  )
print("Fiteando")
model.fit(X, y)
print("Fin del fit")



Fiteando


/home/jose/.venv/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Fin del fit


In [20]:
features = data.drop(['clase_ternaria','clase_peso', 'clase_binaria1','clase_binaria2'], axis=1).columns
importances = model.feature_importances_
feat_importances = pd.DataFrame({'feature': features, 'importance': importances})
feat_importances = feat_importances.sort_values('importance', ascending=False)
print(feat_importances.head(20))

if grabar_importancias:
    if importancias_file.endswith('.gz'):
      feat_importances.to_parquet(os.path.join(dataset_path,  importancias_file), index=False, compression='gzip')
    else:
      feat_importances.to_parquet(os.path.join(dataset_path,  importancias_file), index=False)

                                   feature  importance
50                                mpayroll          11
838             avg_mcomisiones_otras_lag2          10
1011  ratio_mcomisiones_mantenimiento_lag1          10
1073         ratio_Visa_mlimitecompra_lag1          10
777                   avg_mcomisiones_lag1           9
368                           mpayroll_n10           9
280                         ratio_mpayroll           9
290        ratio_mcomisiones_mantenimiento           9
836     avg_mcomisiones_mantenimiento_lag2           9
778                   avg_mcomisiones_lag2           9
49                            cpayroll_trx           8
148                       mtarjeta_consumo           8
1012  ratio_mcomisiones_mantenimiento_lag2           8
203                  avg_mcomisiones_otras           7
173                        avg_mcomisiones           7
105                           ctrx_quarter           7
147                 ctarjeta_transacciones           7
835     av

## Crear variables RF

In [21]:
# Ahora, con todos los datos

cols_fotos=[col for col in data_original.columns if 'foto_mes_lag' in col]
data=data_original.drop(columns=cols_fotos)
y_completo=data['clase_binaria2']
X_completo=data.drop(['clase_ternaria','clase_peso', 'clase_binaria1','clase_binaria2'], axis=1)


In [22]:
# Obtener las hojas para cada observación
leaf_indices = model.predict(X_completo, pred_leaf=True)
leaf_indices.shape

(4901237, 20)

In [23]:
print(model.get_params())

{'boosting_type': 'rf', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 16, 'objective': 'binary', 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'num_iterations': 20, 'min_data_in_leaf': 1000, 'feature_fraction_bynode': 0.2, 'bagging_fraction': 0.6321205588285577, 'bagging_freq': 1, 'feature_fraction': 1, 'max_bin': 31, 'first_metric_only': True, 'boost_from_average': True, 'feature_pre_filter': False, 'force_row_wise': True, 'verbosity': -100, 'min_gain_to_split': 0.0, 'min_sum_hessian_in_leaf': 0.001, 'lambda_l1': 0.0, 'lambda_l2': 0.0, 'pos_bagging_fraction': 1.0, 'neg_bagging_fraction': 1.0, 'is_unbalance': True, 'scale_pos_weight': 1.0, 'drop_rate': 0.1, 'max_drop': 50, 'skip_drop': 0.5, 'extra_trees': False}


In [24]:
# Crear el OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)  # sparse_output=False para obtener un array denso
# Ajustar y transformar los índices de las hojas
leaf_dummies = encoder.fit_transform(leaf_indices)
print (f'Shape de leaf_dummies: {leaf_dummies.shape}')

# Obtener nombres de las columnas para las variables dummy
n_trees = leaf_indices.shape[1]
print (f"Cant de árboles: {n_trees}")
leaf_columns = [f'tree_{i}_leaf_{leaf}' for i in range(n_trees) for leaf in range(encoder.categories_[i].size)]

# Convertir a DataFrame
leaf_dummies_df = pd.DataFrame(leaf_dummies, columns=leaf_columns)
data_extended=data_original.copy()
num_var=0
for campo in leaf_dummies_df.columns:
  num_var+=1
  print (f'{num_var} Agregando campo: {campo}')
  data_extended[campo]=leaf_dummies_df[campo]
print (f'Nuevo shape: {data_extended.shape}')

Shape de leaf_dummies: (4901237, 320)
Cant de árboles: 20
1 Agregando campo: tree_0_leaf_0
2 Agregando campo: tree_0_leaf_1
3 Agregando campo: tree_0_leaf_2
4 Agregando campo: tree_0_leaf_3
5 Agregando campo: tree_0_leaf_4
6 Agregando campo: tree_0_leaf_5
7 Agregando campo: tree_0_leaf_6
8 Agregando campo: tree_0_leaf_7
9 Agregando campo: tree_0_leaf_8
10 Agregando campo: tree_0_leaf_9
11 Agregando campo: tree_0_leaf_10
12 Agregando campo: tree_0_leaf_11
13 Agregando campo: tree_0_leaf_12
14 Agregando campo: tree_0_leaf_13
15 Agregando campo: tree_0_leaf_14
16 Agregando campo: tree_0_leaf_15
17 Agregando campo: tree_1_leaf_0
18 Agregando campo: tree_1_leaf_1
19 Agregando campo: tree_1_leaf_2
20 Agregando campo: tree_1_leaf_3
21 Agregando campo: tree_1_leaf_4
22 Agregando campo: tree_1_leaf_5
23 Agregando campo: tree_1_leaf_6
24 Agregando campo: tree_1_leaf_7
25 Agregando campo: tree_1_leaf_8
26 Agregando campo: tree_1_leaf_9
27 Agregando campo: tree_1_leaf_10
28 Agregando campo: tree_1

/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]
/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]
/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

99 Agregando campo: tree_6_leaf_2
100 Agregando campo: tree_6_leaf_3
101 Agregando campo: tree_6_leaf_4


/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]
/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]


102 Agregando campo: tree_6_leaf_5
103 Agregando campo: tree_6_leaf_6


...

/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]
/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]


316 Agregando campo: tree_19_leaf_11
317 Agregando campo: tree_19_leaf_12


/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]
/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]
/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

318 Agregando campo: tree_19_leaf_13
319 Agregando campo: tree_19_leaf_14
320 Agregando campo: tree_19_leaf_15
Nuevo shape: (4901237, 1625)


/tmp/ipykernel_2158/1647941747.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_extended[campo]=leaf_dummies_df[campo]


## Ver resultado

In [25]:
num_variables_originales=data_original.shape[1]
num_variables_tree = len([col for col in data_extended.columns if col.startswith('tree')])
num_variables_extended=data_extended.shape[1]
print (f'Cant de variables originales: {num_variables_originales}')
print (f'Cant de variables tree: {num_variables_tree}')
print (f'Cant de variables extended: {num_variables_extended}')
print(f'Archivo a grabar: {output_file}')

Cant de variables originales: 1305
Cant de variables tree: 320
Cant de variables extended: 1625
Archivo a grabar: k3__pcf.parquet.gz


In [26]:
# Filtra las columnas que empiezan con 'tree'
tree_columns = [col for col in data_extended.columns if col.startswith('tree')]

# Calcula la suma para cada registro en estas columnas
suma_tree = data_extended[tree_columns].sum(axis=1)

# Muestra el resultado
print(suma_tree)

0          20.0
1          20.0
2          20.0
3          20.0
4          20.0
           ... 
4901232    20.0
4901233    20.0
4901234    20.0
4901235    20.0
4901236    20.0
Length: 4901237, dtype: float64


In [27]:
contabilizar(data_extended, 'Datos extendidos con RF')


Contabilización de la base Datos extendidos con RF


Shape: (4901237, 1625)

\ por mes: 
foto_mes
202109    165644
202108    165442
202107    165152
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
202012    161526
202011    160742
202010    159731
202009    158371
202008    157058
202007    155764
202006    153757
202005    151261
202004    149872
202003    149356
202002    147109
202001    143966
201912    140661
201911    138667
201910    136682
201909    134314
201908    132664
201907    130724
201906    129186
201905    127659
201904    126996
201903    126436
201902    125799
201901    124752
Name: count, dtype: int64


Filas por mes y clase: 
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
201901             688     718    123346
201902             720     688    124391
201903             688     760    124988
201904             759     579    125658
201905             580     660    126419
2

## Grabar salida

In [28]:
data_original.dtypes

numero_de_cliente                 int64
foto_mes                          int64
active_quarter                    int64
cliente_vip                       int64
internet                          int64
                                 ...   
Tarjeta_mpagominimo_n10_lag1    float64
Tarjeta_mpagominimo_n10_lag2    float64
clase_peso                      float64
clase_binaria1                    int64
clase_binaria2                    int64
Length: 1305, dtype: object

In [29]:
data_extended.dtypes

numero_de_cliente      int64
foto_mes               int64
active_quarter         int64
cliente_vip            int64
internet               int64
                      ...   
tree_19_leaf_11      float64
tree_19_leaf_12      float64
tree_19_leaf_13      float64
tree_19_leaf_14      float64
tree_19_leaf_15      float64
Length: 1625, dtype: object

In [30]:
# Grabar el archivo
if output_file.endswith('.gz'):
    data_extended.to_parquet(os.path.join(dataset_path, output_file), index=False, compression='gzip')
else:
    data_extended.to_parquet(os.path.join(dataset_path, output_file), index=False)



In [31]:
print (output_file)

k3__pcf.parquet.gz
